# Quantum VQC Running
## Multiclass Performance with Single Label Training & Prediction

Here are the experiments with the quantum side of things. In this notebook we run the VQC model.

In [ ]:
# For dataset.
from sklearn.datasets import make_multilabel_classification, make_classification
from qiskit_algorithms.utils import algorithm_globals
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC, QSVC
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit_machine_learning.kernels import BaseKernel, TrainableFidelityQuantumKernel

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import pandas as pd
import abc

In [ ]:
# for data split
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap

In [ ]:
from qiskit.circuit.library import RealAmplitudes

----

In [ ]:
class quantumMultiLabelAlgoTrainingVQC:
    """
    Class to train quantum algorithms for multiclass multilabel.
    """
    def __init__(self, no_of_features, no_of_samples, no_of_classes, no_of_labels, no_of_feature_map_reps, no_of_ansatz_reps, models = ['VQC']):
        self.no_of_features = no_of_features
        self.no_of_samples = no_of_samples
        self.no_of_labels = no_of_labels
        self.no_of_classes = no_of_classes
        self.models = models
        self.featuremap_reps = no_of_feature_map_reps
        self.ansatz_reps = no_of_ansatz_reps
        self.objective_func_vals = []
        print(f"the no of classes is {self.no_of_classes}")

    def data_generation(self):
        """
        Generate classification data using sklearn's data generation.\
        """
        n_samples=self.no_of_samples
        n_features=self.no_of_features
        n_classes=self.no_of_classes
        n_labels=self.no_of_labels
        print(n_samples)
        X, y = make_multilabel_classification(n_samples=n_samples, 
                                              n_features=n_features, 
                                              n_classes=n_classes, 
                                              n_labels=n_labels,
                                              random_state=algorithm_globals.random_seed
                                             )
        y_new = np.array([self.conv_to_int(val) for val in y])
        print(X.shape, y.shape, y_new.shape)
        print(X[:5], y[:5], y_new[:5])
        return X, y, y_new

    def int_to_hot(self, x):
        format = '{' + '0:0{:d}b'.format(self.no_of_classes) + '}'
        result = format.format(x)
        list_of_ints = [int(x) for x in result]
        return np.array(list_of_ints)

    def get_mapping(self):
        dictionary = {}
        numbers = list(range(0, 2**self.no_of_classes))
        for i in numbers:
            dictionary[i] = self.int_to_hot(i)
        return dictionary

    def conv_to_int(self, vector):
        val = "".join(vector.astype(str))
        value = int(val, 2)
        return value

    def generate_featuremap(self):
        """ Generating the feature map."""
        feature_map = ZZFeatureMap(feature_dimension=self.no_of_features, reps=self.featuremap_reps)
        return feature_map

    def generate_ansatz(self):
        """Generating ansatz."""
        ansatz = RealAmplitudes(num_qubits=self.no_of_features, reps=self.ansatz_reps)
        return ansatz

    def callback_graph(self, weights, obj_func_eval):
        clear_output(wait=True)
        self.objective_func_vals.append(obj_func_eval)
        plt.title("Objective function value against iteration")
        plt.xlabel("Iteration")
        plt.ylabel("Objective function value")
        plt.plot(range(len(self.objective_func_vals)), self.objective_func_vals)
        plt.show()

    def quantum_training(self):
        X, y, y_new = self.data_generation()
        mapping = self.get_mapping()
        optimizer = COBYLA(maxiter=50)
        model = 'VQC'
        vqc = VQC(
            feature_map=self.generate_featuremap(),
            ansatz=self.generate_ansatz(),
            loss='cross_entropy',
            optimizer=optimizer,
            callback=self.callback_graph,
        )
        
        # clear objective value history
        objective_func_vals = []
        
        start = time.time()
        vqc.fit(X, y_new)
        elapsed = time.time() - start
      
        print(f"Training time: {round(elapsed)} seconds")
        
        # testing the QSVC scores
        predictions = vqc.predict(X)

        # Use list comprehension to create the new array
        print(f"The type of predictions is: {type(predictions)}")
        
        predictions_labels_final = np.array([mapping[val] for val in predictions])
        print(y.shape, predictions_labels_final.shape)
        print(predictions_labels_final[:5])
        accuracy_scores = accuracy_score(y, predictions_labels_final)
        f1_scores = f1_score(y, predictions_labels_final, average='weighted')
        results.setdefault('Model', []).append(model)
        results.setdefault('No of features', []).append(self.no_of_features)
        results.setdefault('No of samples', []).append(self.no_of_samples)
        results.setdefault('No of classes', []).append(self.no_of_classes)
        results.setdefault('No of labels', []).append(self.no_of_labels)
        results.setdefault('No of feature map reps', []).append(self.featuremap_reps)
        results.setdefault('No of ansatz reps', []).append(self.ansatz_reps)
        results.setdefault('Accuracy', []).append(accuracy_scores)
        results.setdefault('F1', []).append(f1_scores)
        results.setdefault('Time taken', []).append(elapsed)

In [ ]:
results = {}
no_of_features = [2,4,6,8,12,16]
no_of_samples = [1024]
no_of_classes = [3]
no_of_labels = [3]
no_of_feature_map_reps = [3]
no_of_ansatz_reps = [3]

In [ ]:
for feature in no_of_features:
    for sample in no_of_samples:
        for featuremap_reps in no_of_feature_map_reps:
            for ansatz_reps in no_of_ansatz_reps:
                for clas in no_of_classes:
                    for lab in no_of_labels:
                        if lab <= clas:
                            
                            training_object = quantumMultiLabelAlgoTrainingVQC(feature, sample, clas, lab, featuremap_reps, ansatz_reps)
                            training_object.quantum_training()

In [ ]:
df = pd.DataFrame(results)

-----